See:
    https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html

    https://pytorch.org/docs/stable/jit.html

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from collections import OrderedDict

In [3]:
from dcgan3D import DCGAN_G

## Just save model w/o checkpoint

The generator needs two parameters, `ngf` and `latent_dim`. Following the setup for https://github.com/FLC-QU-hep/getting_high, they are set to:

In [4]:
ngf = 32
latent_dim = 100

In [5]:
generator = DCGAN_G(ngf, latent_dim)

In [9]:
script_model = torch.jit.script(generator)
print(script_model)

RecursiveScriptModule(
  original_name=DCGAN_G
  (conv1_1): RecursiveScriptModule(original_name=ConvTranspose3d)
  (conv1_100): RecursiveScriptModule(original_name=ConvTranspose3d)
  (main_conv): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=ConvTranspose3d)
    (1): RecursiveScriptModule(original_name=LayerNorm)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=ConvTranspose3d)
    (4): RecursiveScriptModule(original_name=LayerNorm)
    (5): RecursiveScriptModule(original_name=ReLU)
    (6): RecursiveScriptModule(original_name=ConvTranspose3d)
    (7): RecursiveScriptModule(original_name=LayerNorm)
    (8): RecursiveScriptModule(original_name=ReLU)
    (9): RecursiveScriptModule(original_name=ConvTranspose3d)
    (10): RecursiveScriptModule(original_name=ReLU)
  )
)


This yields us a TorchScript Model object that contains the structure of the model that would be readable both by C++ and by Python. So we can save the model.

In [10]:
script_model.save("./gan_wo_checkpoint.pt")

We even have access to a more code-like version of the forward function defined for our dcgan

In [12]:
print(script_model.code)

def forward(self,
    noise: Tensor,
    energy: Tensor) -> Tensor:
  energy_trans = (self.conv1_1).forward(energy, None, )
  noise_trans = (self.conv1_100).forward(noise, None, )
  input = torch.cat([energy_trans, noise_trans], 1)
  x = (self.main_conv).forward(input, )
  return torch.view(x, [-1, 30, 30, 30])

